In [1]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import numpy as np
from sklearn.metrics import mean_squared_error
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import Lasso
import os
import pickle

In [2]:
df= pd.read_csv("dataset/flight_dataset.csv")
df

,Airline,Source,Destination,Total_Stops,Price,Date,Month,Year,Dep_hours,Dep_min,Arrival_hours,Arrival_min,Duration_hours,Duration_min
0,IndiGo,Banglore,New Delhi,0,3897,24,3,2019,22,20,1,10,2,50
1,Air India,Kolkata,Banglore,2,7662,1,5,2019,5,50,13,15,7,25
2,Jet Airways,Delhi,Cochin,2,13882,9,6,2019,9,25,4,25,19,0
3,IndiGo,Kolkata,Banglore,1,6218,12,5,2019,18,5,23,30,5,25
4,IndiGo,Banglore,New Delhi,1,13302,1,3,2019,16,50,21,35,4,45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10678,Air Asia,Kolkata,Banglore,0,4107,9,4,2019,19,55,22,25,2,30
10679,Air India,Kolkata,Banglore,0,4145,27,4,2019,20,45,23,20,2,35
10680,Jet Airways,Banglore,Delhi,0,7229,27,4,2019,8,20,11,20,3,0
10681,Vistara,Banglore,New Delhi,0,12648,1,3,2019,11,30,14,10,2,40


In [52]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Airline         10683 non-null  object
 1   Source          10683 non-null  object
 2   Destination     10683 non-null  object
 3   Total_Stops     10683 non-null  int64 
 4   Price           10683 non-null  int64 
 5   Date            10683 non-null  int64 
 6   Month           10683 non-null  int64 
 7   Year            10683 non-null  int64 
 8   Dep_hours       10683 non-null  int64 
 9   Dep_min         10683 non-null  int64 
 10  Arrival_hours   10683 non-null  int64 
 11  Arrival_min     10683 non-null  int64 
 12  Duration_hours  10683 non-null  int64 
 13  Duration_min    10683 non-null  int64 
dtypes: int64(11), object(3)
memory usage: 1.1+ MB


In [3]:
categorical = ["Airline", "Source", "Destination"]
numerical = ["Total_Stops","Duration_hours","Duration_min"]

In [4]:
df = df[categorical + numerical]
df

,Airline,Source,Destination,Total_Stops,Duration_hours,Duration_min
0,IndiGo,Banglore,New Delhi,0,2,50
1,Air India,Kolkata,Banglore,2,7,25
2,Jet Airways,Delhi,Cochin,2,19,0
3,IndiGo,Kolkata,Banglore,1,5,25
4,IndiGo,Banglore,New Delhi,1,4,45
...,...,...,...,...,...,...
10678,Air Asia,Kolkata,Banglore,0,2,30
10679,Air India,Kolkata,Banglore,0,2,35
10680,Jet Airways,Banglore,Delhi,0,3,0
10681,Vistara,Banglore,New Delhi,0,2,40


In [5]:
df.Duration_hours = df.Duration_hours *60
df

C:\Users\hp\AppData\Local\Temp\ipykernel_22112\4098694595.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.Duration_hours = df.Duration_hours *60


,Airline,Source,Destination,Total_Stops,Duration_hours,Duration_min
0,IndiGo,Banglore,New Delhi,0,120,50
1,Air India,Kolkata,Banglore,2,420,25
2,Jet Airways,Delhi,Cochin,2,1140,0
3,IndiGo,Kolkata,Banglore,1,300,25
4,IndiGo,Banglore,New Delhi,1,240,45
...,...,...,...,...,...,...
10678,Air Asia,Kolkata,Banglore,0,120,30
10679,Air India,Kolkata,Banglore,0,120,35
10680,Jet Airways,Banglore,Delhi,0,180,0
10681,Vistara,Banglore,New Delhi,0,120,40


In [6]:
df["duration"] = df["Duration_hours"] + df["Duration_min"]
df

C:\Users\hp\AppData\Local\Temp\ipykernel_22112\3890359722.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["duration"] = df["Duration_hours"] + df["Duration_min"]


,Airline,Source,Destination,Total_Stops,Duration_hours,Duration_min,duration
0,IndiGo,Banglore,New Delhi,0,120,50,170
1,Air India,Kolkata,Banglore,2,420,25,445
2,Jet Airways,Delhi,Cochin,2,1140,0,1140
3,IndiGo,Kolkata,Banglore,1,300,25,325
4,IndiGo,Banglore,New Delhi,1,240,45,285
...,...,...,...,...,...,...,...
10678,Air Asia,Kolkata,Banglore,0,120,30,150
10679,Air India,Kolkata,Banglore,0,120,35,155
10680,Jet Airways,Banglore,Delhi,0,180,0,180
10681,Vistara,Banglore,New Delhi,0,120,40,160


In [7]:
target = df["duration"].values

In [8]:
df = df.drop(["Duration_hours", "Duration_min", "duration"], axis = 1)
df

,Airline,Source,Destination,Total_Stops
0,IndiGo,Banglore,New Delhi,0
1,Air India,Kolkata,Banglore,2
2,Jet Airways,Delhi,Cochin,2
3,IndiGo,Kolkata,Banglore,1
4,IndiGo,Banglore,New Delhi,1
...,...,...,...,...
10678,Air Asia,Kolkata,Banglore,0
10679,Air India,Kolkata,Banglore,0
10680,Jet Airways,Banglore,Delhi,0
10681,Vistara,Banglore,New Delhi,0


In [9]:
df = df.to_dict(orient = "records")

In [10]:
dv = DictVectorizer()
with open("dict_vectorizer.bin", 'wb') as f_out:
    pickle.dump(dv, f_out)
data_df = dv.fit_transform(df)

In [11]:
dv.feature_names_

['Airline=Air Asia',
 'Airline=Air India',
 'Airline=GoAir',
 'Airline=IndiGo',
 'Airline=Jet Airways',
 'Airline=Jet Airways Business',
 'Airline=Multiple carriers',
 'Airline=Multiple carriers Premium economy',
 'Airline=SpiceJet',
 'Airline=Trujet',
 'Airline=Vistara',
 'Airline=Vistara Premium economy',
 'Destination=Banglore',
 'Destination=Cochin',
 'Destination=Delhi',
 'Destination=Hyderabad',
 'Destination=Kolkata',
 'Destination=New Delhi',
 'Source=Banglore',
 'Source=Chennai',
 'Source=Delhi',
 'Source=Kolkata',
 'Source=Mumbai',
 'Total_Stops']

In [12]:
train_df = data_df[:8000]
test_df = data_df[8000:]
y_train =  target[:8000]
y_test = target[8000:]

In [14]:
#X_train, y_train, X_test, y_test = train_test_split(data_df,target, test_size=0.25, random_state = 32, shuffle = True)

In [13]:
lr = LogisticRegression(solver='liblinear')
lr.fit(train_df, y_train )

LogisticRegression(solver='liblinear')

In [14]:
y_pred = lr.predict(test_df)

In [65]:
rmse = mean_squared_error(y_pred, y_test, squared = False)
rmse

400.98476431787003

In [14]:
import mlflow

In [20]:
mlflow.set_tracking_uri("http://127.0.0.1:5000")

mlflow.set_experiment("S3-deployment-2")

2024/08/03 19:32:21 INFO mlflow.tracking.fluent: Experiment with name 'S3-deployment-2' does not exist. Creating a new experiment.


<Experiment: artifact_location='s3://mlop-zoomcamp-adebayo/3', creation_time=1722709941145, experiment_id='3', last_update_time=1722709941145, lifecycle_stage='active', name='S3-deployment-2', tags={}>

In [21]:

with mlflow.start_run():
    mlflow.set_tag("AI engineer", "Adebayo Abdulganiyu Keji jnr")

    las = Lasso(0.001, max_iter=10000)
    las.fit(train_df, y_train )

    y_pred = las.predict(test_df)
    
    rmse = mean_squared_error(y_pred, y_test, squared = False)



    mlflow.log_metric("rmse", rmse)
    mlflow.sklearn.log_model(las, artifact_path="model")

    mlflow.log_artifact('dict_vectorizer.bin', artifact_path='artifacts')



In [22]:
logged_model = 'runs:/224f5aac05a64508919581fd1889e2ba/model'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)




g:\Anaconda\envs\prefect-ops\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [27]:
import sklearn
print(sklearn.__version__)

1.2.2
